# Phân chia ngẫu nhiên dữ liệu 

In [1]:
import os
import shutil
import random

In [2]:
!pwd

/mnt/DataK/Project/ThyroidCancer/app/process_data


In [3]:
dataset_info = dict()

# change to home project
cwd = os.getcwd()
if cwd[-13:] != "ThyroidCancer":
    os.chdir(cwd[:-16])  
print(os.getcwd())

# Đường dẫn tới thư mục chứa ảnh
image_dir = 'data/processed_data/patch_level/run/'

# Tạo thư mục train, valid, test
train_dir = 'data/processed_data/patch_level/train_valid_test_1/train/'
valid_dir = 'data/processed_data/patch_level/train_valid_test_1/valid/'
test_dir = 'data/processed_data/patch_level/train_valid_test_1/test/'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Tạo thư mục nhãn trong từng tập
labels = ['B2', 'B5', 'B6']
dataset_info['labels'] = labels

for label in labels:
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(valid_dir, label), exist_ok=True)
    os.makedirs(os.path.join(test_dir, label), exist_ok=True)


/mnt/DataK/Project/ThyroidCancer


In [4]:
# Đọc danh sách tất cả các ảnh
all_images = os.listdir(image_dir)
random.shuffle(all_images)

# Tính số lượng ảnh cho mỗi nhãn
num_images = len(all_images)
dataset_info['total number of images'] = num_images
num_train_images = int(0.75 * num_images)
num_valid_images = int(0.10 * num_images)
dataset_info['num_train_images_should_be'] = num_train_images
dataset_info['num_valid_images_should_be'] = num_valid_images
dataset_info['num_test_images_should_be'] = num_images - num_train_images - num_valid_images

data_train_label = {'B2': 0, 'B5': 0, 'B6': 0}
data_valid_label = {'B2': 0, 'B5': 0, 'B6': 0}
data_test_label = {'B2': 0, 'B5': 0, 'B6': 0}

# Cách này có thể không làm đảm bảo cân bằng dữ liệu (nhưng đảm bảo đều ra cả 3 tập)
# Phân chia ảnh vào các thư mục train, valid và test
for i, image in enumerate(all_images):
    label = image[:2]
    src_path = os.path.join(image_dir, image)
    if label in labels:
        if i < num_train_images:
            dst_path = os.path.join(train_dir, label, image)
            data_train_label[label] += 1
        elif num_train_images <= i < (num_train_images + num_valid_images):
            dst_path = os.path.join(valid_dir, label, image)
            data_valid_label[label] += 1
        else:
            dst_path = os.path.join(test_dir, label, image)   
            data_test_label[label] += 1 
        shutil.copy(src_path, dst_path)

dataset_info['data_train_label'] = data_train_label
dataset_info['data_valid_label'] = data_valid_label
dataset_info['data_test_label'] = data_test_label

KeyboardInterrupt: 

In [ ]:
from collections import defaultdict

# Tính số lượng ảnh cho mỗi nhãn
label_counts = defaultdict(int)
for image in all_images:
    label = image[:2]
    if label in labels:
        label_counts[label] += 1
dataset_info['label_counts'] = label_counts

# Tính số lượng ảnh cho mỗi nhãn trong từng tập
num_train_images_per_label = int(num_train_images / len(labels))
num_valid_images_per_label = int(num_valid_images / len(labels))
dataset_info['num_train_images_should_be_per_label'] = num_train_images_per_label
dataset_info['num_valid_images_should_be_per_label'] = num_valid_images_per_label


In [ ]:
print(dataset_info)

{'labels': ['B2', 'B5', 'B6'], 'total number of images': 4942, 'num_train_images_should_be': 3706, 'num_valid_images_should_be': 494, 'num_test_images_should_be': 742, 'data_train_label': {'B2': 184, 'B5': 956, 'B6': 2566}, 'data_valid_label': {'B2': 30, 'B5': 146, 'B6': 318}, 'data_test_label': {'B2': 41, 'B5': 204, 'B6': 497}, 'label_counts': defaultdict(<class 'int'>, {'B6': 3381, 'B5': 1306, 'B2': 255}), 'num_train_images_should_be_per_label': 1235, 'num_valid_images_should_be_per_label': 164}


# Làm cân bằng dữ liệu 

In [ ]:
!pwd

/mnt/DataK/Project/ThyroidCancer


In [ ]:
(2566+318+497) * 3

10143

In [ ]:
import os
import random
from PIL import Image
import shutil

# Đường dẫn đến thư mục chứa các thư mục nhãn
data_dirs = 'data/processed_data/patch_level/train_valid_test/'
# Số lượng ảnh mong muốn cho mỗi nhãn
target_counts = {'train': 2566, 'valid': 318, 'test': 497}
# (2566+318+497) * 3
labels = ['B2', 'B5', 'B6']

# Hàm kiểm tra số lượng ảnh trong thư mục
def count_images_in_directory(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])


In [ ]:
import random
from PIL import Image
import os

def balance_image_counts(directory=data_dirs, target_counts=target_counts, labels=labels):
    for dataset, target_count in target_counts.items():
        dataset_dir = os.path.join(directory, dataset)  # dataset có train, valid, test
        # print(dataset_dir)
        for label in labels:
            # label có B2, B5, B6
            label_dir = os.path.join(dataset_dir, label)
            current_count = count_images_in_directory(label_dir)
            # print(label_dir)
            if current_count < target_count:
                print(f"Balancing images for label {label}...")
                while current_count < target_count:
                    # Chọn ngẫu nhiên một ảnh từ thư mục gốc
                    ref_image = random.choice(os.listdir(label_dir))
                    ref_image_path = os.path.join(label_dir, ref_image)
                    # Tạo tên mới cho ảnh được nhân đôi
                    new_image_name = f"copied_{current_count}.jpg"
                    new_image_path = os.path.join(label_dir, new_image_name)
                    # Mở ảnh
                    image = Image.open(ref_image_path)
                    # Xoay ảnh một góc ngẫu nhiên trong khoảng [0, 90, 180, 270] độ
                    angle = random.choice([0, 90, 180, 270])
                    image = image.rotate(angle)
                    # Lưu ảnh đã xoay
                    image.save(new_image_path)
                    # Cập nhật số lượng ảnh
                    current_count += 1

In [ ]:
balance_image_counts()

Balancing images for label B2...
Balancing images for label B5...
Balancing images for label B2...
Balancing images for label B5...
Balancing images for label B2...
Balancing images for label B5...


In [ ]:
# Check
train_B2_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/train/B2'
train_B2_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/train/B5'
train_B6_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/train/B6'

valid_B2_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/valid/B2'
valid_B5_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/valid/B5'
valid_B6_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/valid/B6'

test_B2_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/test/B2'
test_B5_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/test/B5'
test_B6_path = '/mnt/DataK/Project/ThyroidCancer/data/processed_data/patch_level/train_valid_test/test/B6'

In [ ]:
directories = [
    train_B2_path,
    train_B2_path,
    train_B6_path,
    valid_B2_path,
    valid_B5_path,
    valid_B6_path,
    test_B2_path,
    test_B5_path,
    test_B6_path,
]

for path in directories:
    print(count_images_in_directory(path))

2566
2566
2566
318
318
318
497
497
497
